In [1]:
import sys
sys.path.append("C:/Users/Tobias.Turke/Desktop/Uni/GraphToSequenceNN/Scripts/")
import numpy as np
from keras import backend as K
from keras.models import Model
from keras import regularizers, activations
from keras.layers import Lambda, concatenate, Dense, Dropout, Input, Embedding, LSTM

from keras.utils import plot_model
from keras.callbacks import History

from NetworkHandler.KerasSupportMethods.SupportMethods import IsKerasTensor
from NetworkHandler.KerasSupportMethods.SupportMethods import ConsoleReporter
from NetworkHandler.Neighbouring.NeighbourhoodCollector import Neighbourhood as Nhood
#from NetworkHandler.KerasSupportMethods.ControledBasicOperations import ControledTensorOperations as CTO
from NetworkHandler.Builder.ModelBuilder import ModelBuilder

'''
Great resources 
    => https://theailearner.com/2019/01/25/multi-input-and-multi-output-models-in-keras/
    => https://machinelearningmastery.com/keras-functional-api-deep-learning/
'''  
hops = 3


Using TensorFlow backend.


In [2]:
#
# This is the pipe to create the network easily!
#

hops = 3
in_enc_dim = 100
in_dec_dim = 1000
edge_dim = 19
found_tokens = 1680


builder = ModelBuilder(input_enc_dim=in_enc_dim, 
                       edge_dim=edge_dim, 
                       input_dec_dim=in_dec_dim)

_, graph_embedding_encoder_states = builder.GraphEmbeddingEncoderBuild(hops=hops)

seq_embedding = Embedding(input_dim=found_tokens, 
                          output_dim=in_enc_dim, 
                          input_length=in_dec_dim,
                          name='sequence_embedding')(builder.get_decoder_inputs())

model = builder.GraphEmbeddingDecoderBuild( embedding_layer=seq_embedding,
                                            prev_memory_state=graph_embedding_encoder_states[0],  
                                            prev_carry_state=graph_embedding_encoder_states[1])


model = builder.MakeModel(layers=[model])

builder.CompileModel(model=model)
builder.Summary(model)
builder.Plot(model=model, file_name='model_graph.png')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
features (InputLayer)           (None, 100)          0                                            
__________________________________________________________________________________________________
fw_neighbourhood (InputLayer)   (None, 19)           0                                            
__________________________________________________________________________________________________
bw_neighbourhood (InputLayer)   (None, 19)           0                                            
__________________________________________________________________________________________________
fw_0_lambda_init (Lambda)       (None, 119)          0           features[0][0]                   
                                                                 fw_neighbourhood[0][0]           
__________